In [97]:
from pathlib import Path

dir = Path.resolve('self','YES')

AttributeError: 'str' object has no attribute '_accessor'

In [98]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import MetaData, create_engine
from sqlalchemy import Column, DateTime, Float, Integer, String
from sqlalchemy import and_, case, exists, not_, or_, select
import pyodbc
from ..Models.base import Base
import os
from ..Util.Paths import pathFromCloudAS,appSheetFolderName,excelName,pedidoExcelSheet,UEN,imagesFolderName,version,TIPO_AUDITORIA,UEN,PROCESO,SUBPROCESO,vsMateriales,SUBPROCESOMateriales ,SUBPROCESOParametros,vsParametros

ImportError: attempted relative import with no known parent package

In [70]:
direccion_servidor = 'hp314547'
nombre_bd = 'AuditoriaDeCampo'
nombre_usuario = 'usr_tecnico'
password = 'auditoria'
driver = 'ODBC Driver 17 for SQL Server'

In [71]:
engine=create_engine('mssql+pyodbc://{0}:{1}@{2}/{3}?driver={4}'.format(nombre_usuario,password,direccion_servidor,nombre_bd,driver))

In [72]:
metadata = MetaData()

In [73]:
metadata.reflect(engine, only=['LOGINFO','TmpPremisasOff_HG_Aprov','TmpParamAuditPremOff_HG_Aprov','DimParametros2','FacPremisasOff_HG_Aprov','ParamAuditPremOff_HG_Aprov','DimCampos'])

In [74]:
Base = automap_base(metadata=metadata)

In [75]:
Base.prepare()

In [76]:
session = Session(engine)

In [48]:
query = '''select PEDIDO_ID,COUNT(*) as num_veces
from TmpPremisasOff_HG_Aprov
where ESTADO_GENERAL != 'sin gestion'
GROUP BY PEDIDO_ID
HAVING COUNT(*)>1
'''

In [49]:
result = session.execute(query).fetchall()

In [77]:
DimCampos = Base.classes.DimCampos
TmpTable = Base.classes.TmpPremisasOff_HG_Aprov


In [81]:
def campoCALIDAD_DATA(TmpTable):

    ###### Etiqueta  [CALIDAD_DATA] = 'NO' LOS CAMPOS PROBADOS Y EL RESTO LOS DEJA COMO [CALIDAD_DATA] = 'SI'
    ###### algo falla
    
    with session.begin():
        session.query(TmpTable).filter(
                     TmpTable.ESTADO_GENERAL == 'SIN GESTION').\
                    update({TmpTable.CALIDAD_DATA :'NO APLICA'},
                    synchronize_session=False)
        session.query(TmpTable).filter(or_(and_(
                     TmpTable.ESTADO_GENERAL.in_(('NO AUDITADO','AUDITADO')),
                                    or_(TmpTable.ENVIAR  != 1,
                                        TmpTable.FECHA_AUDITORIA  == None,
                                        TmpTable.HORA_INICIO   == None,
                                        TmpTable.LATITUD_LONGITUD   == None,
                                        TmpTable.HORA_FINALIZACION   == None)),
                                        and_(
                     TmpTable.ESTADO_GENERAL == 'AUDITADO',TmpTable.TECNOLOGIA_AUDIT   == None))
                                        ).\
                    update({TmpTable.CALIDAD_DATA :'NO'},
                    synchronize_session=False)

        session.query(TmpTable).filter(
                     TmpTable.CALIDAD_DATA == None).\
                    update({TmpTable.CALIDAD_DATA :'SI'},
                    synchronize_session=False)
        session.commit()